In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp

from qubic.lib.Qdictionary import qubicDict
from qubic.lib.Instrument.Qacquisition import QubicMultiAcquisitions
from qubic.lib.Qsamplings import get_pointing, equ2gal
from qubic.lib.MapMaking.FrequencyMapMaking.Qspectra_component import CMBModel

%matplotlib inline

In [ ]:
nside = 128

# Initialize QUBIC instance

Build the QUBIC dictionary, mandatory to use the QUBIC functions

In [ ]:
# Build MPI communicator and qubic dict
dictfilename = 'qubic/qubic/dicts/pipeline_demo.dict'
d = qubicDict()
d.read_from_file(dictfilename)

center = equ2gal(d['RA_center'], d['DEC_center'])

d['nf_recon'] = 2
d['MultiBand'] = True
d['nside'] = nside

d['npointings'] = 20
d['synthbeam_kmax'] = 1
nf_sub = d['nf_sub']
d['synthbeam_fraction'] = 1

d['noiseless'] = True
d['photon_noise'] = False
d['use_synthbeam_fits_file'] = False

In [ ]:
d["interp_projection"] = False

In [ ]:
# Build scanning strategy
sampling = get_pointing(d)

In [ ]:
# Number of sub-acquisitions
nsub_max = 54
nsub_list = np.arange(4, nsub_max + 1, 10)

In [ ]:
TOD_list = []
freq_list = []
maps_list = []

np.random.seed(d['seed'])

# Build CMB map
cl_cmb = CMBModel(None).give_cl_cmb(r=0, Alens=1)
cmb_map = hp.synfast(cl_cmb, nside, new=True, verbose=False).T

for insub in nsub_list:
    print(f"Processing {insub} sub-acquisitions")
    input_maps = np.ones((insub, hp.nside2npix(nside), 3)) * cmb_map[None, ...]
    
    # Build acquisition operator
    Qacq = QubicMultiAcquisitions(d, insub, 2, sampling=sampling)
    
    # Build TOD
    tod = []
    map_list = []
    for i in range(insub):
        tod.append(Qacq.H[i](input_maps[i]))
        map_list.append(Qacq.H[i].T(tod[-1]))

    TOD_list.append(np.sum(tod, axis=0))
    maps_list.append(np.mean(map_list, axis=0))
    freq_list.append(np.array(Qacq.allnus))

In [ ]:
for i in range(nsub_list.size):
    plt.plot(np.mean(TOD_list[i], axis=0), label=f"Nsub = {nsub_list[i]}")

plt.title("Mean over detector")
plt.xlabel("Pointing")
plt.ylabel("TOD")
plt.legend(loc='best')

In [ ]:
det_index = 200

for i in range(nsub_list.size):
    plt.plot(TOD_list[i][det_index], label=f"Nsub = {nsub_list[i]}")

plt.title(f"Detector {det_index}")
plt.xlabel("Pointing")
plt.ylabel("TOD")
plt.legend(loc='best')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

for i in range(len(TOD_list)):
    rel_error = np.mean(np.abs((TOD_list[i] - TOD_list[-1]) / TOD_list[-1]))
    
    ax1.plot(nsub_list[i], rel_error, '.')
    ax2.plot(nsub_list[i], rel_error, '.')

ax1.set_xlabel('Nsub')
ax1.set_ylabel(r'|$\frac{H(map)^{max} - H(map)^{i}}{H(map)^{max}}$|')
ax1.set_title('Convergence of H (linear scale)')

ax2.set_xlabel('Nsub')
ax2.set_ylabel(r'|$\frac{H(map)^{max} - H(map)^{i}}{H(map)^{max}}$|')
ax2.set_title('Convergence of H (log scale)')
ax2.set_yscale('log')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

for i in range(len(TOD_list)):
    rel_error = np.mean(np.abs((TOD_list[-1][:, 0] - TOD_list[i][:, 0]) / TOD_list[-1][:, 0]))
    
    ax1.plot(nsub_list[i], rel_error, '.')
    ax2.plot(nsub_list[i], rel_error, '.')

ax1.set_xlabel('Nsub')
ax1.set_ylabel(r'|$\frac{H(map)^{max} - H(map)^{i}}{H(map)^{max}}$|')
ax1.set_title('Convergence of H (linear scale)')

ax2.set_xlabel('Nsub')
ax2.set_ylabel(r'|$\frac{H(map)^{max} - H(map)^{i}}{H(map)^{max}}$|')
ax2.set_title('Convergence of H (log scale)')
ax2.set_yscale('log')

In [ ]:
print(TOD_list[-1].shape)

In [ ]:
plt.figure(figsize=(10,6))
idet = 40
for i in range(len(TOD_list)):
    rel_error = np.mean(np.abs((TOD_list[-1][idet] - TOD_list[i][idet]) / TOD_list[-1][idet]))
    
    plt.plot(nsub_list[i], rel_error, '.')
plt.xlabel('Number of subbands')
plt.ylabel(r'|$\frac{H(map)^{max} - H(map)^{i}}{H(map)^{max}}$|')
plt.yscale('log')
plt.legend()
plt.title(f'Convergence with number of subbands - Nside {nside} - Interpolation {d["interp_projection"]} - Detector {idet}')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
for i in range(len(TOD_list)):
    rel_error = np.mean(np.abs((TOD_list[-1] - TOD_list[i]) / TOD_list[-1]))
    
    plt.plot(nsub_list[i], rel_error, '.')
plt.xlabel('Number of subbands')
plt.ylabel(r'|$\frac{H(map)^{max} - H(map)^{i}}{H(map)^{max}}$|')
plt.yscale('log')
plt.legend()
plt.title(f'Convergence with number of subbands - Nside {nside} - Interpolation {d["interp_projection"]}')
plt.show()